## Bond Data Download

In [7]:
import pickle
from pykrx.website.krx.krxio import KrxWebIo
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [8]:
data_path = 'simulation_data'

class Bond_YTM(KrxWebIo):
    @property
    def bld(self):
        return 'dbms/MDC/STAT/standard/MDCSTAT11402'
    def fetch(self, strtDd: str, endDd: str, bndKindTpCd: str):
        result = self.read(locale="ko_KR", inqTpCd='E', bndKindTpCd=bndKindTpCd, strtDd=strtDd, endDd=endDd)
        return pd.DataFrame(result['output'])

In [9]:
# 채권 데이터를 저장할 Frame 생성
start_date = '2016-01-02'
end_date = '2025-03-20'
with open(f"{data_path}/KS200_MASK.pkl", "rb") as f:
    frame = pickle.load(f) # 열이 시계열
# start_date와 end_date 사이의 데이터만 추출
frame = frame.loc[:, start_date:end_date]

In [10]:
# bndKindTpCd
# 국고채 10년: 3013
# cd 91일: 3013
# 회사채 AA- 무보증 3년: 3009
# 회사채 BB- 무보증 3년: 3010
def fast_fill_frame(frame: pd.DataFrame, bond_df: pd.DataFrame) -> pd.DataFrame:
    # 1) 복사 & numeric 변환
    frame_ = frame.copy().apply(pd.to_numeric, errors='coerce')
    
    # 2) gov10_df의 'LST_ORD_BAS_YD' 시리즈를 frame 컬럼 순서에 맞게 재색인
    #    (컬럼이 완전히 일치한다면 reindex는 생략 가능)
    series = bond_df.loc['LST_ORD_BAS_YD'].reindex(frame_.columns)
    
    # 3) 벡터 브로드캐스트로 한 번에 모든 행을 채우기
    #    DataFrame[:] = 2D numpy array 형태로 할당하면 모든 셀에 일괄 적용됩니다.
    frame_.iloc[:, :] = np.tile(series.values, (len(frame_), 1))
    
    return frame_

def fetch_bond_df(start_date, end_date, bond_type, frame):
    # 1) 원시 데이터 fetch & 날짜 포맷 정리
    raw = Bond_YTM().fetch(
        strtDd=start_date.replace('-', ''), 
        endDd=end_date.replace('-', ''), 
        bndKindTpCd=bond_type
    ).sort_values(by='DISCLS_DD')
    
    raw['DISCLS_DD'] = raw['DISCLS_DD'].str.replace('/', '-')
    raw = raw.set_index('DISCLS_DD')
    
    # 2) 모든 컬럼을 강제 numeric → float (문자열은 NaN 처리)
    raw = raw.apply(pd.to_numeric, errors='coerce')
    
    # 3) 전치하여 날짜가 컬럼이 되도록
    X = raw.transpose()
    
    # 4) target 시계열 D로 리인덱스 & 정렬
    D = frame.columns
    X = X.sort_index(axis=1)
    X = X.reindex(columns=D)
    
    # 5) 보간 및 끝단 채우기
    X = (
        X
        .interpolate(method="linear", axis=1)  # 중간 NaN 선형 보간
        .ffill(axis=1)                         # 앞단 NaN forward-fill
        .bfill(axis=1)                         # 뒷단 NaN backward-fill
    )
    
    # 6) 원본 범위 밖 컬럼(drop)
    orig_dates = raw.index  # 실제 있는 날짜들
    min_date, max_date = orig_dates.min(), orig_dates.max()
    keep = [d for d in D if min_date <= d <= max_date]
    X_final = X.loc[:, keep]
    
    return fast_fill_frame(frame, X_final)

In [11]:
# bond_data_dict={
#     'gov10': '3013', # 국고채 10년 금리
#     'gov3': '3000', # 국고채 3년 금리
#     'rf': '4000', # CD 91일 금리
#     'corp_aa': '3009', # 회사채 AA- 무보증 3년 금리
#     'corp_bb': '3010', # 회사채 BB- 무보증 3년 금리
# }
# for bond_type, bond_code in bond_data_dict.items():
#     print(f"Fetching {bond_type} data...")
#     bond_df = fetch_bond_df(start_date, end_date, bond_code, frame)
#     print(f"Saving {bond_type} data...")
#     with open(f"{data_path}/{bond_type}_bond.pkl", "wb") as f:
#         pickle.dump(bond_df, f)
#     print(f"{bond_type} data saved.")

## Data Load and Portfolio Construction

In [12]:
start_date = '2016-03-30'
end_date = '2025-03-20'
with open(f"{data_path}/KS200_MASK.pkl", 'rb') as f:
    mask_df:pd.DataFrame = pickle.load(f).ffill(axis=1)
    mask_df = mask_df.loc[:, start_date:end_date]
with open(f"{data_path}/Return.pkl", 'rb') as f:
    returns_df = pickle.load(f).ffill(axis=1) * 0.01
    returns_df = returns_df.loc[:, start_date:end_date]
with open(f"{data_path}/MarketCap.pkl", 'rb') as f:
    mc_df = pickle.load(f).ffill(axis=1)
    mc_df = mc_df.loc[:, start_date:end_date]
with open(f"{data_path}/ifrs-full_Equity.pkl", 'rb') as f:
    be_df = pickle.load(f).ffill(axis=1)
    be_df = be_df.loc[:, start_date:end_date]
with open(f"{data_path}/KOSPI_Close.pkl", 'rb') as f:
    kospi_close_df = pickle.load(f).ffill(axis=1)
    kospi_return_df = kospi_close_df.pct_change(axis=1)
    mkt_df = kospi_return_df.loc[:, start_date:end_date]
with open(f"{data_path}/rf_bond.pkl", 'rb') as f:
    rf_df = pickle.load(f).ffill(axis=1) * 0.01
    rf_df = rf_df.loc[:, start_date:end_date]
with open(f"{data_path}/corp_aa_bond.pkl", 'rb') as f:
    corp_aa_df = pickle.load(f).ffill(axis=1) * 0.01
    corp_aa_df = corp_aa_df.loc[:, start_date:end_date]
with open(f"{data_path}/corp_bb_bond.pkl", 'rb') as f:
    corp_bb_df = pickle.load(f).ffill(axis=1) * 0.01
    corp_bb_df = corp_bb_df.loc[:, start_date:end_date]
with open(f"{data_path}/gov10_bond.pkl", 'rb') as f:
    gov10_df = pickle.load(f).ffill(axis=1) * 0.01
    gov10_df = gov10_df.loc[:, start_date:end_date]
with open(f"{data_path}/gov3_bond.pkl", 'rb') as f:
    gov3_df = pickle.load(f).ffill(axis=1) * 0.01
    gov3_df = gov3_df.loc[:, start_date:end_date]
corp_df = (corp_aa_df + corp_bb_df) / 2

In [13]:
import statsmodels.api as sm

R     = returns_df
MC    = mc_df
BE    = be_df 
RF = rf_df / 252 # 일단위 reblancing 시행하므로 252로 나누어 줌.
GOV10   = gov10_df / 252
GOV3 = gov3_df / 252
CORP  = corp_df / 252
MKT   = mkt_df
mask  = mask_df.astype(bool)

dates = R.columns
tickers = R.index

## Regressions for 5 models

In [14]:
import statsmodels.api as sm


# ── 1) 일별 리밸런싱으로 25개 포트폴리오(S1B1…S5B5) 수익 계산 ──
port_rets = {f"S{i}B{j}":[] for i in range(1,6) for j in range(1,6)}

for dt in dates:
    members = mask[dt][mask[dt]].index
    if members.empty: 
        continue

    # size quintile breakpoints
    me = MC.loc[members, dt].dropna()
    sz_br = me.quantile([0.2,0.4,0.6,0.8]).values

    # BE/ME quintile breakpoints
    bm = (BE.loc[members, dt] / MC.loc[members, dt]).dropna()
    bm_br = bm.quantile([0.2,0.4,0.6,0.8]).values

    # 각 포트폴리오
    for i in range(1,6):
        size_idx = me[
            (me >  (sz_br[i-2] if i>1 else -np.inf)) &
            (me <= (sz_br[i-1] if i<5 else  np.inf))
        ].index
        for j in range(1,6):
            bm_idx = bm[
              (bm >  (bm_br[j-2] if j>1 else -np.inf)) &
              (bm <= (bm_br[j-1] if j<5 else  np.inf))
            ].index

            idx = size_idx.intersection(bm_idx)
            name = f"S{i}B{j}"
            if idx.empty:
                ret = np.nan
            else:
                w = MC.loc[idx, dt]
                w = w / w.sum()
                ret = (R.loc[idx, dt] * w).sum()
            port_rets[name].append((dt, ret))

# DataFrame으로 변환
port_df = pd.DataFrame({
    name: pd.Series(dict(port_rets[name]))
    for name in port_rets
}).sort_index(axis=1)
port_df = port_df.interpolate(method='linear', axis=0, limit_direction='both')


# ── 2) SMB, HML 계산 ──
# SMB: size quintile 1 평균 – quintile 5 평균
SMB = (port_df[[f"S1B{j}" for j in range(1,6)]].mean(axis=1) + port_df[[f"S2B{j}" for j in range(1,6)]].mean(axis=1)) \
        - (port_df[[f"S5B{j}" for j in range(1,6)]].mean(axis=1) + port_df[[f"S4B{j}" for j in range(1,6)]].mean(axis=1)) 

# HML: BE quintile 5 평균 – quintile 1 평균
HML = (port_df[[f"S{i}B5" for i in range(1,6)]].mean(axis=1) + port_df[[f"S{i}B4" for i in range(1,6)]].mean(axis=1))\
     - (port_df[[f"S{i}B1" for i in range(1,6)]].mean(axis=1) + port_df[[f"S{i}B2" for i in range(1,6)]].mean(axis=1))

# ── 3) 팩터 DataFrame 생성 ──
factors = pd.DataFrame({
    'MKT_RF': MKT.iloc[0] - RF.iloc[0],
    'SMB':     SMB,
    'HML':     HML,
    'TERM':    GOV10.iloc[0]  - RF.iloc[0],
    'DEF':     CORP.iloc[0] - GOV3.iloc[0],
    'RF':      RF.iloc[0]
}, index=dates).dropna()

# ── 4) 5가지 모형 정의 ──
models = {
    'i_TERM_DEF'      : ['TERM','DEF'],
    'ii_MKT_RF'       : ['MKT_RF'],
    'iii_SMB_HML'     : ['SMB','HML'],
    'iv_MKT_SMB_HML'  : ['MKT_RF','SMB','HML'],
    'v_all_5_factors' : ['MKT_RF','SMB','HML','TERM','DEF'],
}

# ── 5) 25개 포트 × 5 모형 회귀 ──
outputs = {}
for mname, facs in models.items():
    recs = []
    for name in port_df.columns:
        y = (port_df[name] - factors['RF']).dropna()
        common = y.index.intersection(factors.index)
        y = y.loc[common]
        X = factors[facs].loc[common]
        Xc = sm.add_constant(X)

        res = sm.OLS(y, Xc).fit()
        rec = {'Portfolio':name}
        for coef in ['const']+facs:
            rec[f"b_{coef}"] = res.params.get(coef, np.nan)
            rec[f"t_{coef}"] = res.tvalues.get(coef, np.nan)
        rec['R2']   = res.rsquared
        rec['s(e)'] = np.sqrt(res.mse_resid)
        recs.append(rec)

    dfm = pd.DataFrame(recs).set_index('Portfolio')
    tuples = [(int(p[1]), int(p[3])) for p in dfm.index]  # ("S{i}B{j}")
    dfm.index = pd.MultiIndex.from_tuples(tuples, names=['SizeQ','BEQ'])
    outputs[mname] = dfm

# ── 6) 결과 출력 ──
for mname, dfm in outputs.items():
    print(f"\n=== Model {mname} ===")

    # b coefficients
    bc = dfm[[c for c in dfm if c.startswith('b_')]].rename(columns=lambda x: x[2:])
    print("\n-- b coefficients --")
    print(bc.unstack(level='BEQ'))

    # t(b)
    tc = dfm[[c for c in dfm if c.startswith('t_')]].rename(columns=lambda x: x[2:])
    print("\n-- t(b) statistics --")
    print(tc.unstack(level='BEQ'))

    # R²
    print("\n-- R² --")
    print(dfm['R2'].unstack(level='BEQ'))

    # s(e)
    print("\n-- s(e) --")
    print(dfm['s(e)'].unstack(level='BEQ'))



=== Model i_TERM_DEF ===

-- b coefficients --
          const                                               TERM             \
BEQ           1         2         3         4         5          1          2   
SizeQ                                                                           
1     -0.005673 -0.011944 -0.010776 -0.009296 -0.012903  -6.631628   4.686270   
2     -0.008185 -0.006906 -0.011162 -0.010037 -0.010942  -6.032278  11.019237   
3     -0.012373 -0.008186 -0.010367 -0.014434 -0.010652  19.403974 -11.021512   
4     -0.011243 -0.007210 -0.010280 -0.007259 -0.010891 -29.359523  -4.818599   
5     -0.007679 -0.006972 -0.011119 -0.011827 -0.008185 -14.033316 -17.390680   

                                             DEF                        \
BEQ            3          4         5          1          2          3   
SizeQ                                                                    
1      -6.867500 -13.148430 -4.127985  35.320720  75.511021  68.538455   
2      

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors

models = list(outputs.keys())

fig, axes = plt.subplots(1, len(models), figsize=(5*len(models), 4))

# 공통 Normalize 객체 생성
norm = colors.Normalize(vmin=0, vmax=1)

for ax, m in zip(axes, models):
    dfm = outputs[m]
    # BEQ를 열로, SizeQ를 행으로 한 pivot table
    mat = dfm['R2'].unstack(level='BEQ').sort_index()
    
    # vmin/vmax 고정 (또는 norm=norm)
    im = ax.imshow(mat,
                   aspect='auto',
                   cmap='viridis',
                   norm=norm)  
    
    ax.set_title(m, fontsize=10)
    ax.set_xlabel('BE/ME quintile')
    ax.set_ylabel('ME quintile')
    ax.set_xticks(range(mat.shape[1]))
    ax.set_xticklabels(mat.columns)
    ax.set_yticks(range(mat.shape[0]))
    ax.set_yticklabels(mat.index)
    
    # 각 서브플롯마다 컬러바 추가
    cbar = fig.colorbar(im, ax=ax, orientation='vertical',
                        fraction=0.046, pad=0.04)
    cbar.set_label('R²', fontsize=10)

plt.suptitle('R2 Heatmaps', y=1.02, fontsize=20)
plt.tight_layout()
plt.savefig('R2_Heatmaps.png', dpi=600, bbox_inches='tight')
plt.show()


## GRS TEST for 5 Regression Models

In [21]:
from scipy.stats import f as f_dist
from numpy.linalg import pinv

def grs_test(Y, F):
    """
    Gibbons-Ross-Shanken (GRS) test for the null hypothesis that the
    """

    # Setting
    T, K = F.shape[0], F.shape[1]-1
    N = Y.shape[1]
    F_tilde = F.drop(axis=1, labels='ONES') # T x K

    F = np.array(F)
    Y = np.array(Y)
    F_tilde = np.array(F_tilde)

    # Calculation
    theta = pinv(F.T @ F) @ F.T @ Y # (K+1) x N

    alpha = theta[0].reshape(N, 1) # N x 1

    E = Y - F @ theta # T x N
    Z = E.T @ E / (T - K - 1) # N x N
    Z_inv = np.linalg.inv(Z) 
    f = F_tilde.mean(axis=0).reshape(K, 1) # 1 x K
    ones = np.ones((T, 1)) # T x 1
    f = f.reshape(1, K) # 1 x K
    S_f = (1/T) * (F_tilde - ones @ f).T @ (F_tilde - ones @ f) # K x K
    S_f_inv = np.linalg.inv(S_f) # K x K
    a = alpha.T @ Z_inv @ alpha
    b = f @ S_f_inv @ f.T
    f_stat = (T-N-K-1)/N * (a) / (1 + b)
    return float(f_stat)

In [22]:
Y_ex = port_df.subtract(factors['RF'], axis=0)
factors['ONES'] = 1
models = {
    'i_TERM_DEF'      : ['ONES', 'TERM','DEF'],
    'ii_MKT_RF'       : ['ONES', 'MKT_RF'],
    'iii_SMB_HML'     : ['ONES', 'SMB','HML'],
    'iv_MKT_SMB_HML'  : ['ONES', 'MKT_RF','SMB','HML'],
    'v_all_5_factors' : ['ONES', 'MKT_RF','SMB','HML','TERM','DEF'],
}
fstats_dict = {}
for m, flist in models.items():
    F = factors[flist]
    fstats = grs_test(Y_ex, F)
    fstats_dict[m] = fstats

### Bootstrapping

In [24]:
from tqdm import tqdm
np.random.seed(2025)
B = 10_000
p_values = {}
for m, flist in models.items():
    fstat_count = 0
    for b in tqdm(range(B), desc=f"Bootstrap {m}", total=B):
        F= factors[flist]
        T = F.shape[0]
        bootstrap_index = np.random.choice([i for i in range(T)], size=T, replace=True)
        F_bootstrap = F.iloc[bootstrap_index]
        fstats = grs_test(Y_ex, F_bootstrap)
        fstat_count += 1 if fstats < fstats_dict[m] else 0
    p_value = fstat_count / B
    p_values[m] = p_value
print("\n=== GRS Test Results ===")
for m, p_value in p_values.items():
    print(f"{m}_p_obs: {fstats_dict[m]:.4f}")
    print(f"{m}: {p_value:.4f}")


Bootstrap v_all_5_factors: 100%|██████████| 10000/10000 [00:19<00:00, 516.22it/s]


=== GRS Test Results ===
i_TERM_DEF_p_obs: 1.1982
i_TERM_DEF: 0.7513
ii_MKT_RF_p_obs: 4.7128
ii_MKT_RF: 1.0000
iii_SMB_HML_p_obs: 1.3095
iii_SMB_HML: 0.0000
iv_MKT_SMB_HML_p_obs: 4.6360
iv_MKT_SMB_HML: 1.0000
v_all_5_factors_p_obs: 0.7476
v_all_5_factors: 0.1809
